In [1]:
from transformers import AutoConfig, AutoModel, AutoTokenizer

# 载入Tokenizer
tokenizer = AutoTokenizer.from_pretrained("/data2/maliang/zhangkaidong/NLP/chatglm", trust_remote_code=True)


/data1/maliang/miniconda3/envs/simRT1/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [2]:
import os
import torch
config = AutoConfig.from_pretrained("/data2/maliang/zhangkaidong/NLP/chatglm", trust_remote_code=True, pre_seq_len=128)
model = AutoModel.from_pretrained("/data2/maliang/zhangkaidong/NLP/chatglm", config=config, trust_remote_code=True)
prefix_state_dict = torch.load(os.path.join('/data2/maliang/zhangkaidong/NLP/ChatGLM-6B/ptuning/output/adgen-chatglm-6b-pt-128-2e-2/checkpoint-10000', "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /home/maliang_m23/.cache/huggingface/modules/transformers_modules/chatglm/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /home/maliang_m23/.cache/huggingface/modules/transformers_modules/chatglm/quantization_kernels_parallel.c -shared -o /home/maliang_m23/.cache/huggingface/modules/transformers_modules/chatglm/quantization_kernels_parallel.so
Load kernel : /home/maliang_m23/.cache/huggingface/modules/transformers_modules/chatglm/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 32
Using quantization cache
Applying quantization to glm layers


Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at /data2/maliang/zhangkaidong/NLP/chatglm and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
# Comment out the following line if you don't use quantization
model = model.quantize(4)
model = model.half().cuda()
model.transformer.prefix_encoder.float()
model = model.eval()

response, history = model.chat(tokenizer, "你好", history=[])

No compiled kernel found.
Compiling kernels : /home/maliang_m23/.cache/huggingface/modules/transformers_modules/chatglm/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /home/maliang_m23/.cache/huggingface/modules/transformers_modules/chatglm/quantization_kernels_parallel.c -shared -o /home/maliang_m23/.cache/huggingface/modules/transformers_modules/chatglm/quantization_kernels_parallel.so
Load kernel : /home/maliang_m23/.cache/huggingface/modules/transformers_modules/chatglm/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 32


The dtype of attention mask (torch.int64) is not bool


In [8]:
response, history = model.chat(tokenizer, "《机械设计基础》这本书的作者是谁？", history=[])
print(response)
response, history = model.chat(tokenizer, "华瑞国际城的户型有哪些？", history=[])
print(response)
response, history = model.chat(tokenizer, "精弘创立于哪？", history=[])
print(response)


机械设计基础的作者是杨可桢，徐克勤
华瑞国际城的主力户型是95、133平方米两室 三室
精弘的创立于是杭州


In [35]:
response, history = model.chat(tokenizer, "人和网是由谁创立的？", history=[])
print(response)
response, history = model.chat(tokenizer, "有个网络平台叫人和网，以'人脉成就事业'为理念,它是谁创立的", history=history)
print(response)

人和网的创立者是阿里巴巴前创业团队资深成员
人和网的创立者是阿里巴巴前创业团队资深成员


In [ ]:
impo